<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/14_Finans_%C4%B0%C3%A7in_Python_RSI_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphanelerin yüklenmesi
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener
!pip install backtesting


#kütüphanelerin çağırılması
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy
import numpy as np

#Tradingview Kütüphanesini açıyoruz.
tv = TvDatafeed()

#Hisse Adını Tanımlıyoruz.
Hisse = "PGSUS"

#Hisse Verilerini Tradingview Kütüphanesinden çekiyoruz.
data = tv.get_hist(symbol=Hisse, exchange='BIST', interval=Interval.in_daily, n_bars=1000)
print(data)

#Göreceli hareketli Ortalama
def rma(series, length=None):
    """
    Belirli bir kapanış fiyatı serisinin Göreceli Hareketli Ortalama'sını (RMA) hesaplar.
    """
    length = int(length) if length and length > 0 else 10
    alpha = (1.0 / length) if length > 0 else 0.5
    rma = series.ewm(alpha=alpha, min_periods=length).mean()
    return rma


#RSI - Göreceli Güç Endeksi
def rsi(series, length=14):
    """
    Belirli bir serinin Göreceli Güç Endeksi'ni (RSI) hesaplayın.

    Parametreler:
    - series: Fiyat verilerini içeren pandas Serisi.
    - length: RSI periyodunun uzunluğu (varsayılan 14'tür).
    - scalar: RSI değerlerini ayarlamak için kullanılan skaler faktör (varsayılan 100'dür).
    - drift: Fiyat değişiklikleri için dönem sayısı (varsayılan 1'dir).

    Döndürür:
    - Girdi parametrelerine göre hesaplanmış RSI değerlerini içeren pandas Serisi.
    """
    # Fiyat değişikliklerini hesaplayın
    scalar = 100
    drift = 1
    negative = series.diff(drift)
    positive = negative.copy()

    # Pozitif seri için negatifleri 0 yapın
    positive[positive < 0] = 0
    # Negatif seri için pozitifleri 0 yapın
    negative[negative > 0] = 0

    # Ortalama kazançları ve kayıpları hesaplayın
    positive_avg = rma(positive, length=length)
    negative_avg = rma(negative, length=length)

    # RSI'yi hesaplayın
    rsi = scalar * positive_avg / (positive_avg + negative_avg.abs())
    return rsi


#RSI i hesaplatıyoruz.
data['RSI14'] = rsi(data['close'],14)
print(data)

#RSI 30 Yukarı Kesişim Giriş Koşulu
data['Entry'] = data['RSI14'] > 30

#RSI 70 Yukarı Kesişim Çıkış Koşulu
data['Exit'] = data['RSI14'] > 70
print(data)


#Her Zaman kullanacağımız Stratejimizi hazırlıyoruz.
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

#data başlıklarını yeniden isimlendiriyoruz.
data.rename(columns={'open': 'Open', 'high': 'High',
                     'low': 'Low',
                     'close': 'Close',
                     'volume': 'Volume'},
                     inplace=True)

#Backtest yapımızı çağırıyoruz ve sonuçları inceliyoruz.
bt = Backtest(data, Strategy, cash=100000, commission=0.002)
Stats = bt.run()
print(Stats)
print(data.to_string())

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-vacvakmq
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-vacvakmq
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17532 sha256=943ca3762de354167ce7bede038eac2db15ed07b0c99e4df3f68d955149b0cfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ednqv58/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=cfdd8a40ecb6f5623a0fc539e9b5b31bec05e63309f0e

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


                         symbol        open        high         low  \
datetime                                                              
2020-08-05 06:00:00  BIST:PGSUS   10.066271   10.291329    9.861672   
2020-08-06 06:00:00  BIST:PGSUS    9.857580    9.914867    8.957344   
2020-08-07 06:00:00  BIST:PGSUS    8.920516    9.387002    8.224880   
2020-08-10 06:00:00  BIST:PGSUS    9.166035    9.256059    8.797757   
2020-08-11 06:00:00  BIST:PGSUS    9.084196    9.972156    9.084196   
...                         ...         ...         ...         ...   
2024-07-29 06:00:00  BIST:PGSUS  231.100010  233.500000  227.200000   
2024-07-30 06:00:00  BIST:PGSUS  229.100010  232.800000  226.100010   
2024-07-31 06:00:00  BIST:PGSUS  223.899990  224.300000  219.200000   
2024-08-01 06:00:00  BIST:PGSUS  229.000000  236.100010  227.300000   
2024-08-02 06:00:00  BIST:PGSUS  231.100000  233.500000  227.500000   

                          close        volume  
datetime                    